In [56]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim 
geolocator = Nominatim(user_agent="leandro")

In [57]:
geo = pd.read_csv(r'C:\Users\leand\Desktop\Data Science (Henry)\Proyectos\Proyecto_grupal\DS-Proyecto_Grupal_Olist\data\geolocation_dataset_brazil.csv')

In [58]:
#Dropeamos duplicados y renombramos columnas para mantener la relación entre tablas

geo.drop_duplicates(inplace=True)
geo = geo.rename(columns={'geolocation_zip_code_prefix': 'zip_code', 'geolocation_lat':'geolatitud','geolocation_lng':'geolongitud', 'geolocation_city':'city', 'geolocation_state' : 'state'})

In [59]:
#Utilizamos este json para normalizar el campo state

dfj = pd.read_json(r'C:\Users\leand\Desktop\Data Science (Henry)\Proyectos\Proyecto_grupal\DS-Proyecto_Grupal_Olist\data\estados.json')

In [60]:
#Mergeamos ambas tablas
geo = geo.merge(dfj[['nome', 'sigla']], right_on='sigla', left_on='state')

#Renombramos columnas para unificar descripcion de mismo contenido de columna
geo.rename(columns={'nome':'state_name'}, inplace=True)

#Normalizamos mayúsculas
geo['city'] = geo['city'].str.title()

#Eliminamos duplicados por 'zip_code' 
geo = geo.drop_duplicates('zip_code')

#Reseteamos indice
geo = geo.reset_index(drop=True)

#Finalmente, nos quedamos con las columnas que nos servirán a futuro.
geo = geo[['zip_code', 'geolatitud', 'geolongitud', 'city', 'state_name']]

In [62]:
#Reemplazamos aquellas latitudes y longitudes que estén en positivo

geo.loc[geo.geolongitud > 0, 'geolongitud'] = geo.geolongitud * (-1)
geo.loc[geo.geolatitud > 5, 'geolatitud'] = geo.geolatitud * (-1)

In [63]:
# SOLUCIÒN A LOS REGISTROS CON COORDENADAS INCORRECTAS

'CREAMOS MÁSCARA DONDE SE CUMPLEN LAS 3 CONDICIONES'
a = (geo.geolongitud < -73) | (geo.geolongitud > -34) | (geo.geolatitud < -33)

'Para buscar las coordenadas con más precisión'
geo['full_data'] = geo.state_name + ',' + geo.city 

In [64]:
'Traemos las coordenadas con Geopy donde se cumple la máscara'

geo.loc[a, 'gcode'] = geo[a]['full_data'].apply(geolocator.geocode)
geo.loc[a,'lat'] = [g.latitude for g in geo[a]['gcode']]
geo.loc[a,'long'] = [g.longitude for g in geo[a]['gcode']]

In [65]:
'Reemplazamos en columna original'

geo.loc[a, 'geolatitud'] = geo[a]['lat']
geo.loc[a, 'geolongitud'] = geo[a]['long']

In [71]:
# Eliminamos columnas auxiliares

geo.drop(columns= ['gcode', 'lat', 'long', 'full_data'], inplace=True)

In [78]:
geo.to_parquet('geolocation_cleanP')